In [37]:
#importing necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

In [16]:
#initialize the requests and BeautifulSoup object with the Nations League stats link (2022-2023)
scrape_url = "https://fbref.com/en/comps/677/UEFA-Nations-League-Stats"
data = requests.get(scrape_url)
soup = BeautifulSoup(data.text)

#filtering out all tables and selecting the first one on the wbepage(specific to this link)
standings = soup.select('table.stats_table')

qual_nations = [
    "Germany",
    "Albania",
    "Austria",
    "Belgium",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "England",
    "France",
    "Georgia",
    "Hungary",
    "Italy",
    "Netherlands",
    "Poland",
    "Portugal",
    "Romania",
    "Scotland",
    "Serbia",
    "Slovakia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Turkey",
    "Ukraine"
]

nation_urls = []
for table in standings:
    links = table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    for link in links:
        if any(country in link for country in qual_nations):
            nation_urls.append(f"https://fbref.com{link}")

nation_urls

['https://fbref.com/en/squads/66cff10b/2022-2023/Slovakia-Men-Stats',
 'https://fbref.com/en/squads/7158b127/2022-2023/Georgia-Men-Stats',
 'https://fbref.com/en/squads/602d3994/2022-2023/Scotland-Men-Stats',
 'https://fbref.com/en/squads/afa29a3e/2022-2023/Ukraine-Men-Stats',
 'https://fbref.com/en/squads/b44b9eb7/2022-2023/Albania-Men-Stats',
 'https://fbref.com/en/squads/7def9493/2022-2023/Romania-Men-Stats',
 'https://fbref.com/en/squads/1d6f5c9b/2022-2023/Serbia-Men-Stats',
 'https://fbref.com/en/squads/296f69e7/2022-2023/Sweden-Men-Stats',
 'https://fbref.com/en/squads/7b08e376/2022-2023/Croatia-Men-Stats',
 'https://fbref.com/en/squads/29a4e4af/2022-2023/Denmark-Men-Stats',
 'https://fbref.com/en/squads/b1b36dcd/2022-2023/France-Men-Stats',
 'https://fbref.com/en/squads/d5121f10/2022-2023/Austria-Men-Stats',
 'https://fbref.com/en/squads/b561dd30/2022-2023/Spain-Men-Stats',
 'https://fbref.com/en/squads/4a1b4ea8/2022-2023/Portugal-Men-Stats',
 'https://fbref.com/en/squads/81021a

In [17]:
all_data = []

for nation_url in nation_urls:
    
    nation_name = nation_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    
    # Fetch and parse nation data
    nation_data = requests.get(nation_url)
    soup = BeautifulSoup(nation_data.text, 'html.parser')
    matches = pd.read_html(StringIO(nation_data.text), match="Scores & Fixtures")[0]

    # Find and fetch shooting data
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    shooting_links = [l for l in links if l and '/all_comps/shooting/' in l]
    shooting_url = f"https://fbref.com{shooting_links[0]}"
    shooting_data = requests.get(shooting_url)
    shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Find and fetch keeper data
    keeper_links = [l for l in links if l and '/all_comps/keeper/' in l]
    keeper_url = f"https://fbref.com{keeper_links[0]}"
    keeper_data = requests.get(keeper_url)
    keeper = pd.read_html(StringIO(keeper_data.text), match="Goalkeeping")[0]
    keeper.columns = keeper.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Merge data
    ovr_data = matches.merge(shooting[["Date", "Sh", "SoT", "PK", "PKatt"]], on="Date")
    ovr_data = ovr_data.merge(keeper[["Date", "Saves", "CS"]], on="Date")
    ovr_data["Nation"] = nation_name

    all_data.append(ovr_data)

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

# Combine all data
final_data = pd.concat(all_data, ignore_index=True)

In [23]:
#initialize the requests and BeautifulSoup object with the Nations League stats link (2020-2021)
scrape_url = "https://fbref.com/en/comps/677/2020-2021/2020-2021-UEFA-Nations-League-Stats"
data = requests.get(scrape_url)
soup = BeautifulSoup(data.text)

#filtering out all tables and selecting the first one on the wbepage(specific to this link)
standings = soup.select('table.stats_table')

qual_nations = [
    "Germany",
    "Albania",
    "Austria",
    "Belgium",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "England",
    "France",
    "Georgia",
    "Hungary",
    "Italy",
    "Netherlands",
    "Poland",
    "Portugal",
    "Romania",
    "Scotland",
    "Serbia",
    "Slovakia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Turkey",
    "Ukraine"
]

nation_urls = []
for table in standings:
    links = table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    for link in links:
        if any(country in link for country in qual_nations):
            nation_urls.append(f"https://fbref.com{link}")

nation_urls

['https://fbref.com/en/squads/7158b127/2020-2021/Georgia-Men-Stats',
 'https://fbref.com/en/squads/b44b9eb7/2020-2021/Albania-Men-Stats',
 'https://fbref.com/en/squads/d5121f10/2020-2021/Austria-Men-Stats',
 'https://fbref.com/en/squads/7def9493/2020-2021/Romania-Men-Stats',
 'https://fbref.com/en/squads/602d3994/2020-2021/Scotland-Men-Stats',
 'https://fbref.com/en/squads/66cff10b/2020-2021/Slovakia-Men-Stats',
 'https://fbref.com/en/squads/b4ac5e97/2020-2021/Hungary-Men-Stats',
 'https://fbref.com/en/squads/1d6f5c9b/2020-2021/Serbia-Men-Stats',
 'https://fbref.com/en/squads/998c5958/2020-2021/Italy-Men-Stats',
 'https://fbref.com/en/squads/5bb5024a/2020-2021/Netherlands-Men-Stats',
 'https://fbref.com/en/squads/8912dcf0/2020-2021/Poland-Men-Stats',
 'https://fbref.com/en/squads/361422b9/2020-2021/Belgium-Men-Stats',
 'https://fbref.com/en/squads/29a4e4af/2020-2021/Denmark-Men-Stats',
 'https://fbref.com/en/squads/1862c019/2020-2021/England-Men-Stats',
 'https://fbref.com/en/squads/b1

In [24]:
all_data = []

for nation_url in nation_urls:
    
    nation_name = nation_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    
    # Fetch and parse nation data
    nation_data = requests.get(nation_url)
    soup = BeautifulSoup(nation_data.text, 'html.parser')
    matches = pd.read_html(StringIO(nation_data.text), match="Scores & Fixtures")[0]

    # Find and fetch shooting data
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    shooting_links = [l for l in links if l and '/all_comps/shooting/' in l]
    shooting_url = f"https://fbref.com{shooting_links[0]}"
    shooting_data = requests.get(shooting_url)
    shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Find and fetch keeper data
    keeper_links = [l for l in links if l and '/all_comps/keeper/' in l]
    keeper_url = f"https://fbref.com{keeper_links[0]}"
    keeper_data = requests.get(keeper_url)
    keeper = pd.read_html(StringIO(keeper_data.text), match="Goalkeeping")[0]
    keeper.columns = keeper.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Merge data
    ovr_data = matches.merge(shooting[["Date", "Sh", "SoT", "PK", "PKatt"]], on="Date")
    ovr_data = ovr_data.merge(keeper[["Date", "Saves", "CS"]], on="Date")
    ovr_data["Nation"] = nation_name

    all_data.append(ovr_data)

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

# Combine all data
new_data = pd.concat(all_data, ignore_index=True)

In [38]:
#initialize the requests and BeautifulSoup object with the Euro 2020 and qualifiers
scrape_url = "https://fbref.com/en/comps/678/2021/2021-UEFA-Euro-Qualifying-Stats"
data = requests.get(scrape_url)
soup = BeautifulSoup(data.text)

#filtering out all tables and selecting the first one on the wbepage(specific to this link)
standings = soup.select('table.stats_table')

qual_nations = [
    "Germany",
    "Albania",
    "Austria",
    "Belgium",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "England",
    "France",
    "Georgia",
    "Hungary",
    "Italy",
    "Netherlands",
    "Poland",
    "Portugal",
    "Romania",
    "Scotland",
    "Serbia",
    "Slovakia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Turkey",
    "Ukraine"
]

nation_urls = []
for table in standings:
    links = table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    for link in links:
        if any(country in link for country in qual_nations):
            nation_urls.append(f"https://fbref.com{link}")

nation_urls

['https://fbref.com/en/squads/1862c019/2021/England-Men-Stats',
 'https://fbref.com/en/squads/afa29a3e/2021/Ukraine-Men-Stats',
 'https://fbref.com/en/squads/4a1b4ea8/2021/Portugal-Men-Stats',
 'https://fbref.com/en/squads/1d6f5c9b/2021/Serbia-Men-Stats',
 'https://fbref.com/en/squads/c1e40422/2021/Germany-Men-Stats',
 'https://fbref.com/en/squads/5bb5024a/2021/Netherlands-Men-Stats',
 'https://fbref.com/en/squads/81021a70/2021/Switzerland-Men-Stats',
 'https://fbref.com/en/squads/29a4e4af/2021/Denmark-Men-Stats',
 'https://fbref.com/en/squads/7158b127/2021/Georgia-Men-Stats',
 'https://fbref.com/en/squads/7b08e376/2021/Croatia-Men-Stats',
 'https://fbref.com/en/squads/66cff10b/2021/Slovakia-Men-Stats',
 'https://fbref.com/en/squads/b4ac5e97/2021/Hungary-Men-Stats',
 'https://fbref.com/en/squads/b561dd30/2021/Spain-Men-Stats',
 'https://fbref.com/en/squads/296f69e7/2021/Sweden-Men-Stats',
 'https://fbref.com/en/squads/7def9493/2021/Romania-Men-Stats',
 'https://fbref.com/en/squads/8912

In [45]:
euro_data = []

for nation_url in nation_urls:
    
    nation_name = nation_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    
    # Fetch and parse nation data
    nation_data = requests.get(nation_url)
    soup = BeautifulSoup(nation_data.text, 'html.parser')
    matches = pd.read_html(StringIO(nation_data.text), match="Scores & Fixtures")[0]

    # Find and fetch shooting data
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    shooting_links = [l for l in links if l and '/all_comps/shooting/' in l]
    shooting_url = f"https://fbref.com{shooting_links[0]}"
    shooting_data = requests.get(shooting_url)
    shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Find and fetch keeper data
    keeper_links = [l for l in links if l and '/all_comps/keeper/' in l]
    keeper_url = f"https://fbref.com{keeper_links[0]}"
    keeper_data = requests.get(keeper_url)
    keeper = pd.read_html(StringIO(keeper_data.text), match="Goalkeeping")[0]
    keeper.columns = keeper.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Merge data
    ovr_data = matches.merge(shooting[["Date", "Sh", "SoT", "PK", "PKatt"]], on="Date")
    ovr_data = ovr_data.merge(keeper[["Date", "Saves", "CS"]], on="Date")
    ovr_data["Nation"] = nation_name

    euro_data.append(ovr_data)

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

# Combine all data
euro_data = pd.concat(euro_data, ignore_index=True)

In [47]:
euro_data["Nation"] = euro_data["Nation"].str.replace(" Men", "")
euro_data.to_csv("euro_2022.csv")

In [49]:
#initialize the requests and BeautifulSoup object with World Cup matches
scrape_url = "https://fbref.com/en/comps/1/World-Cup-Stats"
data = requests.get(scrape_url)
soup = BeautifulSoup(data.text)

#filtering out all tables and selecting the first one on the wbepage(specific to this link)
standings = soup.select('table.stats_table')

qual_nations = [
    "Germany",
    "Albania",
    "Austria",
    "Belgium",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "England",
    "France",
    "Georgia",
    "Hungary",
    "Italy",
    "Netherlands",
    "Poland",
    "Portugal",
    "Romania",
    "Scotland",
    "Serbia",
    "Slovakia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Turkey",
    "Ukraine"
]

nation_urls = []
for table in standings:
    links = table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    for link in links:
        if any(country in link for country in qual_nations):
            nation_urls.append(f"https://fbref.com{link}")

nation_urls

['https://fbref.com/en/squads/5bb5024a/2022/Netherlands-Men-Stats',
 'https://fbref.com/en/squads/1862c019/2022/England-Men-Stats',
 'https://fbref.com/en/squads/8912dcf0/2022/Poland-Men-Stats',
 'https://fbref.com/en/squads/b1b36dcd/2022/France-Men-Stats',
 'https://fbref.com/en/squads/29a4e4af/2022/Denmark-Men-Stats',
 'https://fbref.com/en/squads/b561dd30/2022/Spain-Men-Stats',
 'https://fbref.com/en/squads/c1e40422/2022/Germany-Men-Stats',
 'https://fbref.com/en/squads/7b08e376/2022/Croatia-Men-Stats',
 'https://fbref.com/en/squads/361422b9/2022/Belgium-Men-Stats',
 'https://fbref.com/en/squads/81021a70/2022/Switzerland-Men-Stats',
 'https://fbref.com/en/squads/1d6f5c9b/2022/Serbia-Men-Stats',
 'https://fbref.com/en/squads/4a1b4ea8/2022/Portugal-Men-Stats',
 'https://fbref.com/en/squads/b1b36dcd/2022/France-Men-Stats',
 'https://fbref.com/en/squads/7b08e376/2022/Croatia-Men-Stats',
 'https://fbref.com/en/squads/5bb5024a/2022/Netherlands-Men-Stats',
 'https://fbref.com/en/squads/186

In [51]:
world_data = []

for nation_url in nation_urls:
    
    nation_name = nation_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    
    # Fetch and parse nation data
    nation_data = requests.get(nation_url)
    soup = BeautifulSoup(nation_data.text, 'html.parser')
    matches = pd.read_html(StringIO(nation_data.text), match="Scores & Fixtures")[0]

    # Find and fetch shooting data
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    shooting_links = [l for l in links if l and '/all_comps/shooting/' in l]
    shooting_url = f"https://fbref.com{shooting_links[0]}"
    shooting_data = requests.get(shooting_url)
    shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Find and fetch keeper data
    keeper_links = [l for l in links if l and '/all_comps/keeper/' in l]
    keeper_url = f"https://fbref.com{keeper_links[0]}"
    keeper_data = requests.get(keeper_url)
    keeper = pd.read_html(StringIO(keeper_data.text), match="Goalkeeping")[0]
    keeper.columns = keeper.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Merge data
    ovr_data = matches.merge(shooting[["Date", "Sh", "SoT", "PK", "PKatt"]], on="Date")
    ovr_data = ovr_data.merge(keeper[["Date", "Saves", "CS"]], on="Date")
    ovr_data["Nation"] = nation_name

    world_data.append(ovr_data)

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

# Combine all data
world_data = pd.concat(world_data, ignore_index=True)

In [53]:
world_data["Nation"] = world_data["Nation"].str.replace(" Men", "")
world_data.to_csv("world_cup.csv")

In [54]:
#initialize the requests and BeautifulSoup object with the Euro 2024 Qualifiers
scrape_url = "https://fbref.com/en/comps/678/UEFA-Euro-Qualifying-Stats"
data = requests.get(scrape_url)
soup = BeautifulSoup(data.text)

#filtering out all tables and selecting the first one on the wbepage(specific to this link)
standings = soup.select('table.stats_table')

qual_nations = [
    "Germany",
    "Albania",
    "Austria",
    "Belgium",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "England",
    "France",
    "Georgia",
    "Hungary",
    "Italy",
    "Netherlands",
    "Poland",
    "Portugal",
    "Romania",
    "Scotland",
    "Serbia",
    "Slovakia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Turkey",
    "Ukraine"
]

nation_urls = []
for table in standings:
    links = table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    for link in links:
        if any(country in link for country in qual_nations):
            nation_urls.append(f"https://fbref.com{link}")

nation_urls

['https://fbref.com/en/squads/b561dd30/Spain-Men-Stats',
 'https://fbref.com/en/squads/602d3994/Scotland-Men-Stats',
 'https://fbref.com/en/squads/7158b127/Georgia-Men-Stats',
 'https://fbref.com/en/squads/b1b36dcd/France-Men-Stats',
 'https://fbref.com/en/squads/5bb5024a/Netherlands-Men-Stats',
 'https://fbref.com/en/squads/1862c019/England-Men-Stats',
 'https://fbref.com/en/squads/998c5958/Italy-Men-Stats',
 'https://fbref.com/en/squads/afa29a3e/Ukraine-Men-Stats',
 'https://fbref.com/en/squads/7b08e376/Croatia-Men-Stats',
 'https://fbref.com/en/squads/b44b9eb7/Albania-Men-Stats',
 'https://fbref.com/en/squads/8912dcf0/Poland-Men-Stats',
 'https://fbref.com/en/squads/361422b9/Belgium-Men-Stats',
 'https://fbref.com/en/squads/d5121f10/Austria-Men-Stats',
 'https://fbref.com/en/squads/296f69e7/Sweden-Men-Stats',
 'https://fbref.com/en/squads/b4ac5e97/Hungary-Men-Stats',
 'https://fbref.com/en/squads/1d6f5c9b/Serbia-Men-Stats',
 'https://fbref.com/en/squads/29a4e4af/Denmark-Men-Stats',


In [55]:
euro_qual = []

for nation_url in nation_urls:
    
    nation_name = nation_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    
    # Fetch and parse nation data
    nation_data = requests.get(nation_url)
    soup = BeautifulSoup(nation_data.text, 'html.parser')
    matches = pd.read_html(StringIO(nation_data.text), match="Scores & Fixtures")[0]

    # Find and fetch shooting data
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    shooting_links = [l for l in links if l and '/all_comps/shooting/' in l]
    shooting_url = f"https://fbref.com{shooting_links[0]}"
    shooting_data = requests.get(shooting_url)
    shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Find and fetch keeper data
    keeper_links = [l for l in links if l and '/all_comps/keeper/' in l]
    keeper_url = f"https://fbref.com{keeper_links[0]}"
    keeper_data = requests.get(keeper_url)
    keeper = pd.read_html(StringIO(keeper_data.text), match="Goalkeeping")[0]
    keeper.columns = keeper.columns.droplevel()

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

    # Merge data
    ovr_data = matches.merge(shooting[["Date", "Sh", "SoT", "PK", "PKatt"]], on="Date")
    ovr_data = ovr_data.merge(keeper[["Date", "Saves", "CS"]], on="Date")
    ovr_data["Nation"] = nation_name

    euro_qual.append(ovr_data)

    # Introduce a delay to avoid exceeding rate limits
    time.sleep(3.5)

# Combine all data
euro_qual = pd.concat(euro_qual, ignore_index=True)

In [56]:
euro_qual

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Match Report,Notes,Sh,SoT,PK,PKatt,Saves,CS,Nation
0,2023-03-25,20:45,UEFA Euro Qualifying,Group stage,Sat,Home,W,3.0,0.0,no Norway,...,Benoît Bastien,Match Report,NaN,13.0,7.0,0,0,2.0,1,Spain Men
1,2023-03-28,19:45,UEFA Euro Qualifying,Group stage,Tue,Away,L,0.0,2.0,sct Scotland,...,Sandro Schärer,Match Report,NaN,8.0,3.0,0,0,1.0,0,Spain Men
2,2023-09-08,20:00,UEFA Euro Qualifying,Group stage,Fri,Away,W,7.0,1.0,ge Georgia,...,Daniel Siebert,Match Report,NaN,33.0,12.0,0,0,1.0,0,Spain Men
3,2023-09-12,20:45,UEFA Euro Qualifying,Group stage,Tue,Home,W,6.0,0.0,cy Cyprus,...,Simone Sozza,Match Report,NaN,28.0,12.0,0,0,1.0,1,Spain Men
4,2023-10-12,20:45,UEFA Euro Qualifying,Group stage,Thu,Home,W,2.0,0.0,sct Scotland,...,Serdar Gözübüyük,Match Report,NaN,18.0,5.0,0,0,1.0,1,Spain Men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2023-10-16,20:45,UEFA Euro Qualifying,Group stage,Mon,Away,W,1.0,0.0,lu Luxembourg,...,José Sánchez,Match Report,NaN,8.0,2.0,0,0,3.0,1,Slovakia Men
225,2023-11-16,20:45,UEFA Euro Qualifying,Group stage,Thu,Home,W,4.0,2.0,is Iceland,...,Craig Pawson,Match Report,NaN,17.0,8.0,1,1,0.0,0,Slovakia Men
226,2023-11-19,20:45,UEFA Euro Qualifying,Group stage,Sun,Away,W,2.0,1.0,ba Bosnia & Herz'na,...,Julian Weinberger,Match Report,NaN,10.0,3.0,0,0,2.0,0,Slovakia Men
227,2024-03-23,18:00,Friendlies (M),Friendlies (M),Sat,Home,L,0.0,2.0,at Austria,...,Trustin Farrugia Cann,Match Report,NaN,NaN,NaN,0,0,NaN,0,Slovakia Men


In [57]:
euro_qual["Nation"] = euro_qual["Nation"].str.replace(" Men", "")
euro_qual.to_csv("euro_qual.csv")